## week01: Intro, Linear models recap, simple ensembles.

## Часть 1. Работа с моделями и ансамблями моделей в задачи классификации.

### 1. Чтение данных.
Воспользуемся [датасетом](https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29), в котором описываются различные автомобили. Будем решать задачу многоклассовой ($k=4$) классификации.
Для удобства, датасет уже преобразован в удобный формат.

In [ ]:
# ! pip install scikit-plot
# ! pip install joblib
# ! wget https://raw.githubusercontent.com/neychev/fall19_madmo_adv/master/week01_Intro_Ensembles/car_data.csv

In [ ]:
from collections import defaultdict
from joblib import Parallel, delayed
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sps
import seaborn as sns
import scikitplot
from sklearn.metrics import accuracy_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.exceptions import DataConversionWarning
from tqdm import tqdm_notebook
import warnings

%matplotlib inline

In [ ]:
dataset = pd.read_csv('car_data.csv', delimiter=',', header=None).values
data = dataset[:, :-1].astype(int)
target = dataset[:, -1]

print(data.shape, target.shape)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.35, random_state=57)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Для первичного анализа может быть полезна библиотека `pandas`. Преобразуем `train` выборку в `pd.DataFrame`.

In [ ]:
X_train_pd = pd.DataFrame(X_train)

# First 15 rows of our dataset.
X_train_pd.head(15)

Воспользовавшись методами `describe` и `info` можно получить полезную информацию о датасете.

In [ ]:
X_train_pd.describe()

In [ ]:
X_train_pd.info()

In [ ]:
# Рассмотрим значения целевой переменной
np.unique(target)

### 2. Работа с данными, построение модели, анализ ошибки.
Выполним основные манипуляции с данными:

#### 2.0. Предобработка данных.
* Какие манипуляции с данными необходимы по вашему мнению?

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Например, в датасете присутствуют категориальные признаки, различные признаки в различных шкалах и пр.

# YOUR CODE HERE

#### 2.1. Базовая логистическая регрессия.
* Подберем оптимальные параметры логистической регресии с помощью кросс-валидации на train-датасете. Небольшого grid/random search'а хватит.

* Построим график ROC-кривой для данного классификатора и оценим точность классификации и f1-score. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline


# YOUR CODE HERE
pipeline = ...

In [ ]:
print("Accuracy score: ", lr_cv.best_score_)

In [ ]:
print("Best model params: ", lr_cv.best_estimator_)

Пакет `scikitplot` позволяет строить красивые `ROC`-кривые (и по-умолчанию удобнее, чем `sklearn.metrics.roc_curve`).

In [ ]:
scikitplot.metrics.plot_roc(y_test, lr_cv.predict_proba(X_test), 
                            figsize=(14, 10))
        

# def plot_roc_auc_curve_multiclass(y_true, y_pred, y_pred_proba, suptitle):
#     sns.set(style="whitegrid", font_scale=1.3)
#     print("- accuracy: {:.3f}\n- f1: {:.3f}".format(
#         accuracy_score(y_test, y_pred),
#         f1_score(y_test, y_pred, average="weighted")
#     ))

# plot_roc_auc_curve_multiclass(y_test, 
#                               lr_cv.predict(X_test), 
#                               lr_cv.predict_proba(X_test), 
#                               "Logistic Regression")

#### 2.2. Зависимость объясненной дисперсии от числа главных компонент.
Воспользуемся методом главных компонент (PCA). 

Применим его к train-части данных.
Затем построим график зависимости объясненной дисперсии (explained variance ratio) от количества главных компонент.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

# YOUR CODE HERE

#### 2.3. Преобразование с помощью PCA.
Выберем оптимальное число компонент. Чем должен быть обусловлен наш выбор?

Используя эти главные компоненты, преобразуем train и test выборки (используя методы `fit` и `transform`).

In [ ]:
pca = PCA(n_components=<N BEST COMPONENTS>)

X_train_proj = # YOUR CODE HERE pca.fit_transform(X_train)
X_test_proj = # YOUR CODE HEREpca.transform(X_test)

#### 2.4. Логистическая регрессия над преобразованными данными.
* Подберем оптимальные параметры логистической регресии с помощью кросс-валидации на преобразованном train-датасете.

* Снова построим график ROC-кривой для полученных классификаторов, оценим точность классификации и f1-score. 

In [ ]:
# YOUR CODE HERE

#### 2.5. Решающее дерево.
Рассмотрим поведение решающего дерева на исходных и преобразованных данных. Будем варьировать лишь один параметр - максимальную глубину дерева. 

* С помощью кросс-валидации подберем оптимальный параметр `max_depth` и оцените на исходных и преобразованных данных те же метрики, что и выше.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# YOUR CODE HERE

In [ ]:
print(accuracy_score(dt_cv.predict(X_train), y_train))

Судя по `accuracy` на обучающей выборке, дерево могло переобучиться. Рассмотрим данные после применения PCA.

In [ ]:
# YOUR CODE HERE

#### 2.6. Bagging.
Перейдем к ансамблям. 

Для построения ансамбля воспользуемся Bagging'ом с числом алгоритмов от 2 до 50. В качестве первого семейства базовых алгоримтов будем использовать линейные модели (т.е. логистическую регрессию), в качестве второго - решающие деревья. 

*Пояснение: Будем строить ансамбль только из моделей из одного семейства, т.е. логистическая регрессия не смешивается с решающими деревьями.*

Для этого можно сгенерировать с помощью метода bootstrap 50 подвыборок из `train` выборки (того же размера, что и исходная), обучить логистическую регрессию и решающее дерево с оптимальными параметрами из предыдущего пункта на каждой из подвыборок и затем усреднить предсказания k моделей.

*Hint: в sklearn может найтись подходящая функция, которая облегчит вам реализацию данного пункта.*

* Построим график качества классификации и f1-score на `train` и `test` датасетах в зависимости от числа алгоритмов, вошедших в ансамбль.

* Проанализируем график. Какое количество моделей стоит использовать? Как вы думаете, являются ли параметры решающего дерева, подобранные в предыдущем пункте оптимальными в данном случае?

In [ ]:
from sklearn.ensemble import BaggingClassifier

# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

А теперь построим ансамбль решающих деревьев.

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

#### 2.7. Random Forest.
Теперь воспользуемся `sklearn`-реализацией алгоритма Random Forest. 

* Построим аналогичные графики для него (изменяя число деревьев от 1 до 50). Остальные параметры можно оставить по умолчанию.

* Проанализируем полученные результаты. Каким получилось оптимальное число деревьев. Как оно соотносится с оптимальным числом деревьев и линейных моделей в bagging'е из предыдущего пункта?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# YOUR CODE HERE


In [ ]:
# YOUR CODE HERE